In [ ]:
%pip install --upgrade pyspark pandas pyarrow plotly nbformat

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
 
spark = SparkSession.builder.getOrCreate()
spark.version

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/12 15:09:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


'3.4.1'

In [19]:
spark.stop()

In [2]:
spark.getActiveSession()

In [2]:
my_grocery_list = [
    ["Banana", 2, 1.74],
    ["Apple", 4, 2.04],
    ["Carrot", 1, 1.09],
    ["Cake", 1, 10.99],
]

df_grocery_list = spark.createDataFrame(
    my_grocery_list, ["Item", "Quantity", "Price"]
)

df_grocery_list.show()
df_grocery_list.printSchema()

+------+--------+-----+
|  Item|Quantity|Price|
+------+--------+-----+
|Banana|       2| 1.74|
| Apple|       4| 2.04|
|Carrot|       1| 1.09|
|  Cake|       1|10.99|
+------+--------+-----+

root
 |-- Item: string (nullable = true)
 |-- Quantity: long (nullable = true)
 |-- Price: double (nullable = true)



Pandas API example

In [11]:
df_grocery_list.pandas_api().plot.bar(x="Item", y="Price")

## Graphical Exploratory Data Analysis

In [3]:
url = "https://applications.crtc.gc.ca/OpenData/Television%20Logs/BroadcastLogs_2018_Q3_M8.CSV"
data_path = '../../../DataAnalysisWithPythonAndPySpark-Data/broadcast_logs'
file_name = 'BroadcastLogs_2018_Q3_M8.CSV'
filepath = f'{data_path}/{file_name}'

In [17]:
# Download CSV file from url and save it to the local file system into specified path
import urllib.request
urllib.request.urlretrieve(url, data_path + '/' + file_name)

('../../../DataAnalysisWithPythonAndPySpark-Data/broadcast_logs/BroadcastLogs_2018_Q3_M8.CSV',
 <http.client.HTTPMessage at 0xfffe2c95fdf0>)

In [3]:
!ls -lh {data_path}

total 1.1G
-rw-rw-r--. 1 opc opc 994M Jul  2 15:49  BroadcastLogs_2018_Q3_M8.CSV
-rw-rw-r--. 1 opc opc  33M Mar  8 18:02  BroadcastLogs_2018_Q3_M8_sample.CSV
-rw-rw-r--. 1 opc opc  18K Mar  8 18:02  Call_Signs.csv
-rw-rw-r--. 1 opc opc  71K Mar  8 18:02 'data dictionary.doc'
drwxrwxr-x. 2 opc opc 4.0K Mar  8 18:02  ReferenceTables


In [4]:
!head -n 5 {data_path}/BroadcastLogs_2018_Q3_M8_sample.CSV

BroadcastLogID|LogServiceID|LogDate|SequenceNO|AudienceTargetAgeID|AudienceTargetEthnicID|CategoryID|ClosedCaptionID|CountryOfOriginID|DubDramaCreditID|EthnicProgramID|ProductionSourceID|ProgramClassID|FilmClassificationID|ExhibitionID|Duration|EndTime|LogEntryDate|ProductionNO|ProgramTitle|StartTime|Subtitle|NetworkAffiliationID|SpecialAttentionID|BroadcastOriginPointID|CompositionID|Producer1|Producer2|Language1|Language2
1196192316|3157|2018-08-01|1|4||13|3|3|||10|19||2|02:00:00.0000000|08:00:00.0000000|2018-08-01|A39082|Newlywed and Dead|06:00:00.0000000||||||||94|
1196192317|3157|2018-08-01|2||||1|||||20|||00:00:30.0000000|06:13:45.0000000|2018-08-01||15-SPECIALTY CHANNELS-Canadian Generic|06:13:15.0000000|||||||||
1196192318|3157|2018-08-01|3||||1|||||3|||00:00:15.0000000|06:14:00.0000000|2018-08-01||3-PROCTER & GAMBLE INC-Anti-Perspirant 3rd|06:13:45.0000000|||||||||
1196192319|3157|2018-08-01|4||||1|||||3|||00:00:15.0000000|06:14:15.0000000|2018-08-01||12-CREDIT KARMA-Bank/Cred

In [15]:
# pysparl specify date columns and format in the csv file
logs_df = spark.read.csv(
    filepath,
    sep="|",
    header=True,
    inferSchema=True,
    dateFormat="yyyy-MM-dd",
    timestampFormat="yyyy-MM-dd HH:mm:ss",
)

In [16]:
logs_df.printSchema()

root
 |-- BroadcastLogID: integer (nullable = true)
 |-- LogServiceID: integer (nullable = true)
 |-- LogDate: date (nullable = true)
 |-- SequenceNO: integer (nullable = true)
 |-- AudienceTargetAgeID: integer (nullable = true)
 |-- AudienceTargetEthnicID: integer (nullable = true)
 |-- CategoryID: integer (nullable = true)
 |-- ClosedCaptionID: integer (nullable = true)
 |-- CountryOfOriginID: integer (nullable = true)
 |-- DubDramaCreditID: integer (nullable = true)
 |-- EthnicProgramID: integer (nullable = true)
 |-- ProductionSourceID: integer (nullable = true)
 |-- ProgramClassID: integer (nullable = true)
 |-- FilmClassificationID: integer (nullable = true)
 |-- ExhibitionID: integer (nullable = true)
 |-- Duration: string (nullable = true)
 |-- EndTime: string (nullable = true)
 |-- LogEntryDate: date (nullable = true)
 |-- ProductionNO: string (nullable = true)
 |-- ProgramTitle: string (nullable = true)
 |-- StartTime: string (nullable = true)
 |-- Subtitle: string (nullable 

In [18]:
logs_df.show(5)

+--------------+------------+----------+----------+-------------------+----------------------+----------+---------------+-----------------+----------------+---------------+------------------+--------------+--------------------+------------+----------------+----------------+------------+------------+--------------------+----------------+--------+--------------------+------------------+----------------------+-------------+---------+---------+---------+---------+
|BroadcastLogID|LogServiceID|   LogDate|SequenceNO|AudienceTargetAgeID|AudienceTargetEthnicID|CategoryID|ClosedCaptionID|CountryOfOriginID|DubDramaCreditID|EthnicProgramID|ProductionSourceID|ProgramClassID|FilmClassificationID|ExhibitionID|        Duration|         EndTime|LogEntryDate|ProductionNO|        ProgramTitle|       StartTime|Subtitle|NetworkAffiliationID|SpecialAttentionID|BroadcastOriginPointID|CompositionID|Producer1|Producer2|Language1|Language2|
+--------------+------------+----------+----------+-------------------

In [13]:
logs_df_1 = (spark.read
           .option("sep", "|")
           .option("header", True)
           .option("inferSchema", True)
           .option("timestampFormat", "HH:mm:ss.SS")
           .option("dateFormat", "yyyy-MM-dd")
           .option("preferDate", True)
           .csv(filepath)
)

In [14]:
logs_df_1.show(5)

+--------------+------------+----------+----------+-------------------+----------------------+----------+---------------+-----------------+----------------+---------------+------------------+--------------+--------------------+------------+----------------+----------------+------------+------------+--------------------+----------------+--------+--------------------+------------------+----------------------+-------------+---------+---------+---------+---------+
|BroadcastLogID|LogServiceID|   LogDate|SequenceNO|AudienceTargetAgeID|AudienceTargetEthnicID|CategoryID|ClosedCaptionID|CountryOfOriginID|DubDramaCreditID|EthnicProgramID|ProductionSourceID|ProgramClassID|FilmClassificationID|ExhibitionID|        Duration|         EndTime|LogEntryDate|ProductionNO|        ProgramTitle|       StartTime|Subtitle|NetworkAffiliationID|SpecialAttentionID|BroadcastOriginPointID|CompositionID|Producer1|Producer2|Language1|Language2|
+--------------+------------+----------+----------+-------------------

In [15]:
logs_df_1.printSchema()

root
 |-- BroadcastLogID: integer (nullable = true)
 |-- LogServiceID: integer (nullable = true)
 |-- LogDate: date (nullable = true)
 |-- SequenceNO: integer (nullable = true)
 |-- AudienceTargetAgeID: integer (nullable = true)
 |-- AudienceTargetEthnicID: integer (nullable = true)
 |-- CategoryID: integer (nullable = true)
 |-- ClosedCaptionID: integer (nullable = true)
 |-- CountryOfOriginID: integer (nullable = true)
 |-- DubDramaCreditID: integer (nullable = true)
 |-- EthnicProgramID: integer (nullable = true)
 |-- ProductionSourceID: integer (nullable = true)
 |-- ProgramClassID: integer (nullable = true)
 |-- FilmClassificationID: integer (nullable = true)
 |-- ExhibitionID: integer (nullable = true)
 |-- Duration: string (nullable = true)
 |-- EndTime: string (nullable = true)
 |-- LogEntryDate: date (nullable = true)
 |-- ProductionNO: string (nullable = true)
 |-- ProgramTitle: string (nullable = true)
 |-- StartTime: string (nullable = true)
 |-- Subtitle: string (nullable 

### 4.4 The basics of data manipulation: Selecting, dropping, renaming, ordering, diagnosing

In [7]:
logs_df.count()

7169318

In [36]:
logs_df.select("BroadcastLogID", "LogServiceID", "LogDate").show(5)

+--------------+------------+----------+
|BroadcastLogID|LogServiceID|   LogDate|
+--------------+------------+----------+
|    1196192316|        3157|2018-08-01|
|    1196192317|        3157|2018-08-01|
|    1196192318|        3157|2018-08-01|
|    1196192319|        3157|2018-08-01|
|    1196192320|        3157|2018-08-01|
+--------------+------------+----------+
only showing top 5 rows



In [12]:
logs_df.select(*["BroadCastLogID", "LogServiceID", "LogDate"]).show(5)

+--------------+------------+----------+
|BroadCastLogID|LogServiceID|   LogDate|
+--------------+------------+----------+
|    1196192316|        3157|2018-08-01|
|    1196192317|        3157|2018-08-01|
|    1196192318|        3157|2018-08-01|
|    1196192319|        3157|2018-08-01|
|    1196192320|        3157|2018-08-01|
+--------------+------------+----------+
only showing top 5 rows



Peeking at the data frame in chunks of three columns

In [26]:
import numpy as np

column_split = np.array_split(
np.array(logs_df.columns), len(logs_df.columns) // 8
)

# print(column_split)

for columns in column_split:
    logs_df.select(*columns).show(5)

+--------------+------------+----------+----------+-------------------+----------------------+----------+---------------+-----------------+----------------+
|BroadcastLogID|LogServiceID|   LogDate|SequenceNO|AudienceTargetAgeID|AudienceTargetEthnicID|CategoryID|ClosedCaptionID|CountryOfOriginID|DubDramaCreditID|
+--------------+------------+----------+----------+-------------------+----------------------+----------+---------------+-----------------+----------------+
|    1196192316|        3157|2018-08-01|         1|                  4|                  null|        13|              3|                3|            null|
|    1196192317|        3157|2018-08-01|         2|               null|                  null|      null|              1|             null|            null|
|    1196192318|        3157|2018-08-01|         3|               null|                  null|      null|              1|             null|            null|
|    1196192319|        3157|2018-08-01|         4|       

In [37]:
logs_df = logs_df.drop("BroadcastLogID", "SequenceNO")

print("BroadcastLogID" in logs_df.columns)  # => False
print("SequenceNo" in logs_df.columns)  # => False

False
False


#### Select * except variant

In [38]:
logs_df = logs_df.select(
    *[x for x in logs_df.columns if x not in ["BroadcastLogID", "SequenceNO"]]
)

In [39]:
logs_df.show(5)

+------------+----------+-------------------+----------------------+----------+---------------+-----------------+----------------+---------------+------------------+--------------+--------------------+------------+----------------+----------------+------------+------------+--------------------+----------------+--------+--------------------+------------------+----------------------+-------------+---------+---------+---------+---------+
|LogServiceID|   LogDate|AudienceTargetAgeID|AudienceTargetEthnicID|CategoryID|ClosedCaptionID|CountryOfOriginID|DubDramaCreditID|EthnicProgramID|ProductionSourceID|ProgramClassID|FilmClassificationID|ExhibitionID|        Duration|         EndTime|LogEntryDate|ProductionNO|        ProgramTitle|       StartTime|Subtitle|NetworkAffiliationID|SpecialAttentionID|BroadcastOriginPointID|CompositionID|Producer1|Producer2|Language1|Language2|
+------------+----------+-------------------+----------------------+----------+---------------+-----------------+---------

In [49]:
# select columns by index
logs_df.select(*logs_df.columns[-3:]).show(5)

+---------+---------+---------+
|Producer2|Language1|Language2|
+---------+---------+---------+
|     null|       94|     null|
|     null|     null|     null|
|     null|     null|     null|
|     null|     null|     null|
|     null|     null|     null|
+---------+---------+---------+
only showing top 5 rows



In [ ]:
# drop last 3 columns
logs_df.drop(*logs_df.columns[-3:])

4.4.3 Creating what’s not there: New columns with withColumn()

In [51]:
logs_df.select(F.col("Duration")).show(5)
print(logs_df.select(F.col("Duration")).dtypes)

+----------------+
|        Duration|
+----------------+
|02:00:00.0000000|
|00:00:30.0000000|
|00:00:15.0000000|
|00:00:15.0000000|
|00:00:15.0000000|
+----------------+
only showing top 5 rows

[('Duration', 'string')]


In [20]:
logs_df.select(
    F.col("Duration"),
    F.col("Duration").substr(1, 2).cast("int").alias("Hours"),
    F.col("Duration").substr(4, 2).cast("int").alias("Minutes"),
    F.col("Duration").substr(7, 2).cast("int").alias("Seconds"),
).show(5)

+----------------+-----+-------+-------+
|        Duration|Hours|Minutes|Seconds|
+----------------+-----+-------+-------+
|02:00:00.0000000|    2|      0|      0|
|00:00:30.0000000|    0|      0|     30|
|00:00:15.0000000|    0|      0|     15|
|00:00:15.0000000|    0|      0|     15|
|00:00:15.0000000|    0|      0|     15|
+----------------+-----+-------+-------+
only showing top 5 rows



In [6]:
logs_df.select(
    F.col("Duration"),
    (F.col("Duration").substr(1, 2).cast("int") * 60 * 60
    + F.col("Duration").substr(4, 2).cast("int") * 60
    + F.col("Duration").substr(7, 2).cast("int")).alias("DurationInSeconds"),
).show(5)

+----------------+-----------------+
|        Duration|DurationInSeconds|
+----------------+-----------------+
|02:00:00.0000000|             7200|
|00:00:30.0000000|               30|
|00:00:15.0000000|               15|
|00:00:15.0000000|               15|
|00:00:15.0000000|               15|
+----------------+-----------------+
only showing top 5 rows



In [17]:
logs_df = logs_df.withColumn(
    "DurationInSeconds",
    (F.col("Duration").substr(1, 2).cast("int") * 60 * 60
    + F.col("Duration").substr(4, 2).cast("int") * 60
    + F.col("Duration").substr(7, 2).cast("int")),
)

logs_df.show(5)

+--------------+------------+----------+----------+-------------------+----------------------+----------+---------------+-----------------+----------------+---------------+------------------+--------------+--------------------+------------+----------------+----------------+------------+------------+--------------------+----------------+--------+--------------------+------------------+----------------------+-------------+---------+---------+---------+---------+-----------------+
|BroadcastLogID|LogServiceID|   LogDate|SequenceNO|AudienceTargetAgeID|AudienceTargetEthnicID|CategoryID|ClosedCaptionID|CountryOfOriginID|DubDramaCreditID|EthnicProgramID|ProductionSourceID|ProgramClassID|FilmClassificationID|ExhibitionID|        Duration|         EndTime|LogEntryDate|ProductionNO|        ProgramTitle|       StartTime|Subtitle|NetworkAffiliationID|SpecialAttentionID|BroadcastOriginPointID|CompositionID|Producer1|Producer2|Language1|Language2|DurationInSeconds|
+--------------+------------+-----

In [18]:
logs_df.printSchema()

root
 |-- BroadcastLogID: integer (nullable = true)
 |-- LogServiceID: integer (nullable = true)
 |-- LogDate: date (nullable = true)
 |-- SequenceNO: integer (nullable = true)
 |-- AudienceTargetAgeID: integer (nullable = true)
 |-- AudienceTargetEthnicID: integer (nullable = true)
 |-- CategoryID: integer (nullable = true)
 |-- ClosedCaptionID: integer (nullable = true)
 |-- CountryOfOriginID: integer (nullable = true)
 |-- DubDramaCreditID: integer (nullable = true)
 |-- EthnicProgramID: integer (nullable = true)
 |-- ProductionSourceID: integer (nullable = true)
 |-- ProgramClassID: integer (nullable = true)
 |-- FilmClassificationID: integer (nullable = true)
 |-- ExhibitionID: integer (nullable = true)
 |-- Duration: string (nullable = true)
 |-- EndTime: string (nullable = true)
 |-- LogEntryDate: date (nullable = true)
 |-- ProductionNO: string (nullable = true)
 |-- ProgramTitle: string (nullable = true)
 |-- StartTime: string (nullable = true)
 |-- Subtitle: string (nullable 

4.4.4 Tidying our data frame: Renaming and reordering columns

In [19]:
logs_df = logs_df.withColumnRenamed("DurationInSeconds", "duration_seconds")

In [20]:
logs_df.printSchema()

root
 |-- BroadcastLogID: integer (nullable = true)
 |-- LogServiceID: integer (nullable = true)
 |-- LogDate: date (nullable = true)
 |-- SequenceNO: integer (nullable = true)
 |-- AudienceTargetAgeID: integer (nullable = true)
 |-- AudienceTargetEthnicID: integer (nullable = true)
 |-- CategoryID: integer (nullable = true)
 |-- ClosedCaptionID: integer (nullable = true)
 |-- CountryOfOriginID: integer (nullable = true)
 |-- DubDramaCreditID: integer (nullable = true)
 |-- EthnicProgramID: integer (nullable = true)
 |-- ProductionSourceID: integer (nullable = true)
 |-- ProgramClassID: integer (nullable = true)
 |-- FilmClassificationID: integer (nullable = true)
 |-- ExhibitionID: integer (nullable = true)
 |-- Duration: string (nullable = true)
 |-- EndTime: string (nullable = true)
 |-- LogEntryDate: date (nullable = true)
 |-- ProductionNO: string (nullable = true)
 |-- ProgramTitle: string (nullable = true)
 |-- StartTime: string (nullable = true)
 |-- Subtitle: string (nullable 

In [21]:
# Batch lowercasing using the toDF() method
logs_df.toDF(*[x.lower() for x in logs_df.columns]).printSchema()

root
 |-- broadcastlogid: integer (nullable = true)
 |-- logserviceid: integer (nullable = true)
 |-- logdate: date (nullable = true)
 |-- sequenceno: integer (nullable = true)
 |-- audiencetargetageid: integer (nullable = true)
 |-- audiencetargetethnicid: integer (nullable = true)
 |-- categoryid: integer (nullable = true)
 |-- closedcaptionid: integer (nullable = true)
 |-- countryoforiginid: integer (nullable = true)
 |-- dubdramacreditid: integer (nullable = true)
 |-- ethnicprogramid: integer (nullable = true)
 |-- productionsourceid: integer (nullable = true)
 |-- programclassid: integer (nullable = true)
 |-- filmclassificationid: integer (nullable = true)
 |-- exhibitionid: integer (nullable = true)
 |-- duration: string (nullable = true)
 |-- endtime: string (nullable = true)
 |-- logentrydate: date (nullable = true)
 |-- productionno: string (nullable = true)
 |-- programtitle: string (nullable = true)
 |-- starttime: string (nullable = true)
 |-- subtitle: string (nullable 

In [23]:
# Sort columns alphabetically
logs_df.select(sorted(logs_df.columns)).show(5)

+-------------------+----------------------+--------------+----------------------+----------+---------------+-------------+-----------------+----------------+----------------+----------------+---------------+------------+--------------------+---------+---------+----------+------------+------------+--------------------+---------+---------+------------+------------------+--------------+--------------------+----------+------------------+----------------+--------+----------------+
|AudienceTargetAgeID|AudienceTargetEthnicID|BroadcastLogID|BroadcastOriginPointID|CategoryID|ClosedCaptionID|CompositionID|CountryOfOriginID|DubDramaCreditID|        Duration|         EndTime|EthnicProgramID|ExhibitionID|FilmClassificationID|Language1|Language2|   LogDate|LogEntryDate|LogServiceID|NetworkAffiliationID|Producer1|Producer2|ProductionNO|ProductionSourceID|ProgramClassID|        ProgramTitle|SequenceNO|SpecialAttentionID|       StartTime|Subtitle|duration_seconds|
+-------------------+---------------

4.4.5 Diagnosing a data frame with describe() and summary()

In [ ]:
# Describe the dataframe columns one by one
for column in sorted(logs_df.columns):
    print(f"Column: {column}")
    logs_df.describe(column).show()

In [26]:
# Describe the dataframe columns all at once
logs_df.describe().show()

+-------+--------------------+------------------+------------------+-------------------+----------------------+-----------------+-------------------+------------------+------------------+------------------+------------------+-----------------+--------------------+------------------+----------------+----------------+-----------------+--------------------+----------------+--------------------+--------------------+------------------+----------------------+------------------+---------+---------+-----------------+------------------+------------------+
|summary|      BroadcastLogID|      LogServiceID|        SequenceNO|AudienceTargetAgeID|AudienceTargetEthnicID|       CategoryID|    ClosedCaptionID| CountryOfOriginID|  DubDramaCreditID|   EthnicProgramID|ProductionSourceID|   ProgramClassID|FilmClassificationID|      ExhibitionID|        Duration|         EndTime|     ProductionNO|        ProgramTitle|       StartTime|            Subtitle|NetworkAffiliationID|SpecialAttentionID|BroadcastOrig

In [32]:
# Sumary of the dataframe columns one by one
for column in sorted(logs_df.columns[15:16]):
    print(f"Column: {column}")
    logs_df.select(column).summary().show()

Column: Duration


+-------+----------------+
|summary|        Duration|
+-------+----------------+
|  count|         7099405|
|   mean|            null|
| stddev|            null|
|    min|00:00:00.0000000|
|    25%|            null|
|    50%|            null|
|    75%|            null|
|    max|18:00:00.0000000|
+-------+----------------+



In [33]:
logs_df.select("duration_seconds").summary().show()

+-------+------------------+
|summary|  duration_seconds|
+-------+------------------+
|  count|           7099405|
|   mean|127.77428883124712|
| stddev| 604.5953472183393|
|    min|                 0|
|    25%|                15|
|    50%|                15|
|    75%|                30|
|    max|             64800|
+-------+------------------+



In [34]:
logs_df.select("duration_seconds").summary("count", "min", "max", "10%", "90%").show()

+-------+----------------+
|summary|duration_seconds|
+-------+----------------+
|  count|         7099405|
|    min|               0|
|    max|           64800|
|    10%|              13|
|    90%|              30|
+-------+----------------+



In [38]:
logs_df.select("duration_seconds").count()

7169318

In [39]:
logs_raw = spark.read.csv(
    filepath,
)

In [40]:
logs_raw.show(5)

+--------------------+
|                 _c0|
+--------------------+
|BroadcastLogID|Lo...|
|1196192316|3157|2...|
|1196192317|3157|2...|
|1196192318|3157|2...|
|1196192319|3157|2...|
+--------------------+
only showing top 5 rows



In [41]:
logs_raw.printSchema()

root
 |-- _c0: string (nullable = true)



In [44]:
# Create dataframe with column names not ending with ID
logs_clean = logs_df.select(
    *[x for x in logs_df.columns if not x.endswith("ID")]
)

logs_clean.show(5)

+----------+----------+----------------+----------------+------------+------------+--------------------+----------------+--------+---------+---------+---------+---------+----------------+
|   LogDate|SequenceNO|        Duration|         EndTime|LogEntryDate|ProductionNO|        ProgramTitle|       StartTime|Subtitle|Producer1|Producer2|Language1|Language2|duration_seconds|
+----------+----------+----------------+----------------+------------+------------+--------------------+----------------+--------+---------+---------+---------+---------+----------------+
|2018-08-01|         1|02:00:00.0000000|08:00:00.0000000|  2018-08-01|      A39082|   Newlywed and Dead|06:00:00.0000000|    null|     null|     null|       94|     null|            7200|
|2018-08-01|         2|00:00:30.0000000|06:13:45.0000000|  2018-08-01|        null|15-SPECIALTY CHAN...|06:13:15.0000000|    null|     null|     null|     null|     null|              30|
|2018-08-01|         3|00:00:15.0000000|06:14:00.0000000|  2